# Simple Models

## Load Data

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
0,0,A,I,A,B,B,BI,A,S,Q,...,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0
1,1,A,I,A,A,E,BI,K,W,AD,...,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,0.493729,0
2,2,A,K,A,A,E,BI,A,E,BM,...,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0
3,3,A,K,A,C,E,BI,A,Y,AD,...,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,0.934242,0
4,4,A,I,G,B,E,BI,C,G,Q,...,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,0.328960,1


In [3]:
cat_cols = [x for x in train.columns if x.startswith('cat')]
cat_cols

['cat0',
 'cat1',
 'cat2',
 'cat3',
 'cat4',
 'cat5',
 'cat6',
 'cat7',
 'cat8',
 'cat9',
 'cat10',
 'cat11',
 'cat12',
 'cat13',
 'cat14',
 'cat15',
 'cat16',
 'cat17',
 'cat18']

In [4]:
x_train = train.drop(columns=['id','target'])
y_train = train['target']

In [5]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True)

## Submit

In [6]:
def submit(model, refit = True):
    if refit:
        model.fit(x_train, y_train)
    pred = model.predict_proba(test.drop(columns='id'))[:,1]
    submission = pd.DataFrame({'id':test['id'], 'target':pred})
    return submission

## Default Lightgbm Model

In [7]:
from categorical_transform import CategoricalTransform
from lightgbm.sklearn import LGBMClassifier
from sklearn.pipeline import Pipeline

In [8]:
lightgbm_pipe = Pipeline([("trans", CategoricalTransform(cat_cols)),
                          ("lgbm", LGBMClassifier(n_jobs = -2))])

In [9]:
from sklearn.model_selection import cross_validate
scores = cross_validate(lightgbm_pipe, X=x_train, y=y_train, cv=kfold, return_train_score = True,
                         scoring='roc_auc')

In [10]:
scores

{'fit_time': array([2.02213287, 1.95802832, 1.99723625, 1.92733455, 1.90357113]),
 'score_time': array([0.2170949 , 0.2159884 , 0.2181468 , 0.21693897, 0.21817183]),
 'test_score': array([0.89198701, 0.89040374, 0.89260234, 0.88969634, 0.89184532]),
 'train_score': array([0.90545169, 0.9056793 , 0.90525984, 0.90585901, 0.90548301])}

In [11]:
scores['test_score'].mean()

0.891306951043863

In [12]:
submission = submit(lightgbm_pipe)
submission.head()

,id,target
0,5,0.140532
1,6,0.368300
2,8,0.030049
3,9,0.177666
4,11,0.112767


In [13]:
submission.to_csv('submission_lightgbm_default.csv', index=False)

## Default CatBoost

In [14]:
from categorical_transform import IntegerCategoricalTransform
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline

In [15]:
cbc = CatBoostClassifier(cat_features=cat_cols, thread_count=6)
catboost_pipe = Pipeline([("trans", IntegerCategoricalTransform(cat_cols)),
                          ("catboost", cbc)])

In [16]:
from sklearn.model_selection import cross_validate
scores = cross_validate(catboost_pipe, X=x_train, y=y_train, cv=kfold, return_train_score = True,
                         scoring='roc_auc')

Learning rate set to 0.106975
0:	learn: 0.6229004	total: 301ms	remaining: 5m
1:	learn: 0.5677715	total: 511ms	remaining: 4m 14s
2:	learn: 0.5250745	total: 697ms	remaining: 3m 51s
3:	learn: 0.4941072	total: 877ms	remaining: 3m 38s
4:	learn: 0.4703077	total: 1.09s	remaining: 3m 36s
5:	learn: 0.4522534	total: 1.27s	remaining: 3m 29s
6:	learn: 0.4360962	total: 1.44s	remaining: 3m 23s
7:	learn: 0.4244002	total: 1.64s	remaining: 3m 23s
8:	learn: 0.4151844	total: 1.84s	remaining: 3m 22s
9:	learn: 0.4080645	total: 2.03s	remaining: 3m 21s
10:	learn: 0.4019788	total: 2.23s	remaining: 3m 20s
11:	learn: 0.3969190	total: 2.44s	remaining: 3m 20s
12:	learn: 0.3926514	total: 2.63s	remaining: 3m 20s
13:	learn: 0.3890135	total: 2.85s	remaining: 3m 20s
14:	learn: 0.3859540	total: 3.08s	remaining: 3m 21s
15:	learn: 0.3836020	total: 3.27s	remaining: 3m 21s
16:	learn: 0.3817514	total: 3.51s	remaining: 3m 22s
17:	learn: 0.3793751	total: 3.74s	remaining: 3m 24s
18:	learn: 0.3775027	total: 3.98s	remaining: 3m 

158:	learn: 0.3470400	total: 35.2s	remaining: 3m 6s
159:	learn: 0.3469874	total: 35.4s	remaining: 3m 5s
160:	learn: 0.3469105	total: 35.6s	remaining: 3m 5s
161:	learn: 0.3468275	total: 35.8s	remaining: 3m 5s
162:	learn: 0.3467631	total: 36s	remaining: 3m 5s
163:	learn: 0.3466809	total: 36.2s	remaining: 3m 4s
164:	learn: 0.3466090	total: 36.5s	remaining: 3m 4s
165:	learn: 0.3465152	total: 36.7s	remaining: 3m 4s
166:	learn: 0.3464204	total: 36.9s	remaining: 3m 4s
167:	learn: 0.3463224	total: 37.2s	remaining: 3m 4s
168:	learn: 0.3462365	total: 37.4s	remaining: 3m 3s
169:	learn: 0.3461805	total: 37.6s	remaining: 3m 3s
170:	learn: 0.3461368	total: 37.9s	remaining: 3m 3s
171:	learn: 0.3460557	total: 38.1s	remaining: 3m 3s
172:	learn: 0.3459827	total: 38.3s	remaining: 3m 3s
173:	learn: 0.3459033	total: 38.5s	remaining: 3m 2s
174:	learn: 0.3458604	total: 38.7s	remaining: 3m 2s
175:	learn: 0.3458056	total: 39s	remaining: 3m 2s
176:	learn: 0.3457408	total: 39.2s	remaining: 3m 2s
177:	learn: 0.34

315:	learn: 0.3388334	total: 1m 10s	remaining: 2m 32s
316:	learn: 0.3387872	total: 1m 10s	remaining: 2m 31s
317:	learn: 0.3387490	total: 1m 10s	remaining: 2m 31s
318:	learn: 0.3387125	total: 1m 10s	remaining: 2m 31s
319:	learn: 0.3386742	total: 1m 11s	remaining: 2m 31s
320:	learn: 0.3386265	total: 1m 11s	remaining: 2m 30s
321:	learn: 0.3386092	total: 1m 11s	remaining: 2m 30s
322:	learn: 0.3385825	total: 1m 11s	remaining: 2m 30s
323:	learn: 0.3385479	total: 1m 12s	remaining: 2m 30s
324:	learn: 0.3384998	total: 1m 12s	remaining: 2m 30s
325:	learn: 0.3384563	total: 1m 12s	remaining: 2m 29s
326:	learn: 0.3384022	total: 1m 12s	remaining: 2m 29s
327:	learn: 0.3384011	total: 1m 12s	remaining: 2m 29s
328:	learn: 0.3383693	total: 1m 13s	remaining: 2m 29s
329:	learn: 0.3382987	total: 1m 13s	remaining: 2m 28s
330:	learn: 0.3382678	total: 1m 13s	remaining: 2m 28s
331:	learn: 0.3382234	total: 1m 13s	remaining: 2m 28s
332:	learn: 0.3382012	total: 1m 13s	remaining: 2m 28s
333:	learn: 0.3381504	total:

468:	learn: 0.3333595	total: 1m 44s	remaining: 1m 57s
469:	learn: 0.3333062	total: 1m 44s	remaining: 1m 57s
470:	learn: 0.3332926	total: 1m 44s	remaining: 1m 57s
471:	learn: 0.3332509	total: 1m 44s	remaining: 1m 57s
472:	learn: 0.3332084	total: 1m 45s	remaining: 1m 57s
473:	learn: 0.3331789	total: 1m 45s	remaining: 1m 56s
474:	learn: 0.3331359	total: 1m 45s	remaining: 1m 56s
475:	learn: 0.3330946	total: 1m 45s	remaining: 1m 56s
476:	learn: 0.3330609	total: 1m 46s	remaining: 1m 56s
477:	learn: 0.3330432	total: 1m 46s	remaining: 1m 56s
478:	learn: 0.3330024	total: 1m 46s	remaining: 1m 55s
479:	learn: 0.3329655	total: 1m 46s	remaining: 1m 55s
480:	learn: 0.3329421	total: 1m 46s	remaining: 1m 55s
481:	learn: 0.3329109	total: 1m 47s	remaining: 1m 55s
482:	learn: 0.3328726	total: 1m 47s	remaining: 1m 54s
483:	learn: 0.3328136	total: 1m 47s	remaining: 1m 54s
484:	learn: 0.3328028	total: 1m 47s	remaining: 1m 54s
485:	learn: 0.3327799	total: 1m 48s	remaining: 1m 54s
486:	learn: 0.3327651	total:

621:	learn: 0.3288313	total: 2m 18s	remaining: 1m 24s
622:	learn: 0.3287944	total: 2m 18s	remaining: 1m 23s
623:	learn: 0.3287860	total: 2m 18s	remaining: 1m 23s
624:	learn: 0.3287579	total: 2m 19s	remaining: 1m 23s
625:	learn: 0.3287399	total: 2m 19s	remaining: 1m 23s
626:	learn: 0.3287166	total: 2m 19s	remaining: 1m 23s
627:	learn: 0.3286838	total: 2m 19s	remaining: 1m 22s
628:	learn: 0.3286650	total: 2m 20s	remaining: 1m 22s
629:	learn: 0.3286341	total: 2m 20s	remaining: 1m 22s
630:	learn: 0.3285930	total: 2m 20s	remaining: 1m 22s
631:	learn: 0.3285678	total: 2m 20s	remaining: 1m 22s
632:	learn: 0.3285401	total: 2m 21s	remaining: 1m 21s
633:	learn: 0.3285152	total: 2m 21s	remaining: 1m 21s
634:	learn: 0.3284896	total: 2m 21s	remaining: 1m 21s
635:	learn: 0.3284533	total: 2m 21s	remaining: 1m 21s
636:	learn: 0.3284272	total: 2m 22s	remaining: 1m 20s
637:	learn: 0.3284039	total: 2m 22s	remaining: 1m 20s
638:	learn: 0.3283712	total: 2m 22s	remaining: 1m 20s
639:	learn: 0.3283528	total:

...

355:	learn: 0.3360221	total: 1m 18s	remaining: 2m 21s
356:	learn: 0.3359902	total: 1m 18s	remaining: 2m 21s
357:	learn: 0.3359561	total: 1m 18s	remaining: 2m 21s
358:	learn: 0.3359244	total: 1m 19s	remaining: 2m 21s
359:	learn: 0.3358856	total: 1m 19s	remaining: 2m 20s
360:	learn: 0.3358374	total: 1m 19s	remaining: 2m 20s
361:	learn: 0.3357766	total: 1m 19s	remaining: 2m 20s
362:	learn: 0.3357188	total: 1m 19s	remaining: 2m 20s
363:	learn: 0.3356914	total: 1m 20s	remaining: 2m 20s
364:	learn: 0.3356453	total: 1m 20s	remaining: 2m 19s
365:	learn: 0.3356267	total: 1m 20s	remaining: 2m 19s
366:	learn: 0.3356121	total: 1m 20s	remaining: 2m 19s
367:	learn: 0.3355788	total: 1m 21s	remaining: 2m 19s
368:	learn: 0.3355309	total: 1m 21s	remaining: 2m 19s
369:	learn: 0.3354899	total: 1m 21s	remaining: 2m 18s
370:	learn: 0.3354452	total: 1m 21s	remaining: 2m 18s
371:	learn: 0.3354071	total: 1m 22s	remaining: 2m 18s
372:	learn: 0.3353522	total: 1m 22s	remaining: 2m 18s
373:	learn: 0.3353037	total:

508:	learn: 0.3307749	total: 1m 52s	remaining: 1m 48s
509:	learn: 0.3307632	total: 1m 53s	remaining: 1m 48s
510:	learn: 0.3307348	total: 1m 53s	remaining: 1m 48s
511:	learn: 0.3307197	total: 1m 53s	remaining: 1m 48s
512:	learn: 0.3306921	total: 1m 53s	remaining: 1m 48s
513:	learn: 0.3306677	total: 1m 53s	remaining: 1m 47s
514:	learn: 0.3306322	total: 1m 54s	remaining: 1m 47s
515:	learn: 0.3305768	total: 1m 54s	remaining: 1m 47s
516:	learn: 0.3305399	total: 1m 54s	remaining: 1m 47s
517:	learn: 0.3305069	total: 1m 54s	remaining: 1m 46s
518:	learn: 0.3304715	total: 1m 55s	remaining: 1m 46s
519:	learn: 0.3304393	total: 1m 55s	remaining: 1m 46s
520:	learn: 0.3304158	total: 1m 55s	remaining: 1m 46s
521:	learn: 0.3303721	total: 1m 55s	remaining: 1m 46s
522:	learn: 0.3303314	total: 1m 56s	remaining: 1m 45s
523:	learn: 0.3303200	total: 1m 56s	remaining: 1m 45s
524:	learn: 0.3302658	total: 1m 56s	remaining: 1m 45s
525:	learn: 0.3302230	total: 1m 56s	remaining: 1m 45s
526:	learn: 0.3301768	total:

661:	learn: 0.3265907	total: 2m 27s	remaining: 1m 15s
662:	learn: 0.3265618	total: 2m 27s	remaining: 1m 14s
663:	learn: 0.3265539	total: 2m 27s	remaining: 1m 14s
664:	learn: 0.3265222	total: 2m 27s	remaining: 1m 14s
665:	learn: 0.3264692	total: 2m 27s	remaining: 1m 14s
666:	learn: 0.3264450	total: 2m 28s	remaining: 1m 13s
667:	learn: 0.3264017	total: 2m 28s	remaining: 1m 13s
668:	learn: 0.3263827	total: 2m 28s	remaining: 1m 13s
669:	learn: 0.3263461	total: 2m 28s	remaining: 1m 13s
670:	learn: 0.3263240	total: 2m 29s	remaining: 1m 13s
671:	learn: 0.3263029	total: 2m 29s	remaining: 1m 12s
672:	learn: 0.3262880	total: 2m 29s	remaining: 1m 12s
673:	learn: 0.3262720	total: 2m 29s	remaining: 1m 12s
674:	learn: 0.3262503	total: 2m 30s	remaining: 1m 12s
675:	learn: 0.3262273	total: 2m 30s	remaining: 1m 12s
676:	learn: 0.3262024	total: 2m 30s	remaining: 1m 11s
677:	learn: 0.3261689	total: 2m 30s	remaining: 1m 11s
678:	learn: 0.3261395	total: 2m 30s	remaining: 1m 11s
679:	learn: 0.3261029	total:

817:	learn: 0.3225552	total: 3m 1s	remaining: 40.5s
818:	learn: 0.3225062	total: 3m 2s	remaining: 40.3s
819:	learn: 0.3224703	total: 3m 2s	remaining: 40s
820:	learn: 0.3224437	total: 3m 2s	remaining: 39.8s
821:	learn: 0.3224050	total: 3m 2s	remaining: 39.6s
822:	learn: 0.3224035	total: 3m 3s	remaining: 39.4s
823:	learn: 0.3223933	total: 3m 3s	remaining: 39.1s
824:	learn: 0.3223813	total: 3m 3s	remaining: 38.9s
825:	learn: 0.3223592	total: 3m 3s	remaining: 38.7s
826:	learn: 0.3223157	total: 3m 3s	remaining: 38.5s
827:	learn: 0.3223145	total: 3m 4s	remaining: 38.2s
828:	learn: 0.3222894	total: 3m 4s	remaining: 38s
829:	learn: 0.3222507	total: 3m 4s	remaining: 37.8s
830:	learn: 0.3222328	total: 3m 4s	remaining: 37.6s
831:	learn: 0.3222006	total: 3m 5s	remaining: 37.4s
832:	learn: 0.3221829	total: 3m 5s	remaining: 37.1s
833:	learn: 0.3221596	total: 3m 5s	remaining: 36.9s
834:	learn: 0.3221337	total: 3m 5s	remaining: 36.7s
835:	learn: 0.3221205	total: 3m 5s	remaining: 36.5s
836:	learn: 0.32

973:	learn: 0.3190233	total: 3m 36s	remaining: 5.79s
974:	learn: 0.3190218	total: 3m 36s	remaining: 5.56s
975:	learn: 0.3189887	total: 3m 37s	remaining: 5.34s
976:	learn: 0.3189709	total: 3m 37s	remaining: 5.12s
977:	learn: 0.3189298	total: 3m 37s	remaining: 4.89s
978:	learn: 0.3189052	total: 3m 37s	remaining: 4.67s
979:	learn: 0.3188631	total: 3m 38s	remaining: 4.45s
980:	learn: 0.3188589	total: 3m 38s	remaining: 4.23s
981:	learn: 0.3188319	total: 3m 38s	remaining: 4s
982:	learn: 0.3188099	total: 3m 38s	remaining: 3.78s
983:	learn: 0.3187949	total: 3m 38s	remaining: 3.56s
984:	learn: 0.3187805	total: 3m 39s	remaining: 3.34s
985:	learn: 0.3187586	total: 3m 39s	remaining: 3.12s
986:	learn: 0.3187567	total: 3m 39s	remaining: 2.89s
987:	learn: 0.3187353	total: 3m 39s	remaining: 2.67s
988:	learn: 0.3186972	total: 3m 40s	remaining: 2.45s
989:	learn: 0.3186658	total: 3m 40s	remaining: 2.23s
990:	learn: 0.3186260	total: 3m 40s	remaining: 2s
991:	learn: 0.3186028	total: 3m 40s	remaining: 1.78s

In [17]:
scores

{'fit_time': array([233.40593195, 231.35192394, 235.1822381 , 233.06129527,
        231.72220802]),
 'score_time': array([2.08055997, 2.09843397, 2.0771594 , 2.07698655, 2.08308935]),
 'test_score': array([0.89411112, 0.8946864 , 0.89484735, 0.89678675, 0.89062138]),
 'train_score': array([0.91219141, 0.91113875, 0.91167955, 0.91154099, 0.91220843])}

In [18]:
scores['test_score'].mean()

0.8942106009845272

In [51]:
submission = submit(catboost_pipe)
submission.head()

Learning rate set to 0.11767
0:	learn: 0.6139339	total: 329ms	remaining: 5m 28s
1:	learn: 0.5567925	total: 619ms	remaining: 5m 8s
2:	learn: 0.5154786	total: 916ms	remaining: 5m 4s
3:	learn: 0.4828273	total: 1.17s	remaining: 4m 50s
4:	learn: 0.4588615	total: 1.41s	remaining: 4m 40s
5:	learn: 0.4406216	total: 1.65s	remaining: 4m 34s
6:	learn: 0.4269822	total: 1.88s	remaining: 4m 26s
7:	learn: 0.4167466	total: 2.13s	remaining: 4m 24s
8:	learn: 0.4081431	total: 2.39s	remaining: 4m 23s
9:	learn: 0.4010793	total: 2.64s	remaining: 4m 21s
10:	learn: 0.3956042	total: 2.87s	remaining: 4m 17s
11:	learn: 0.3911239	total: 3.16s	remaining: 4m 19s
12:	learn: 0.3877442	total: 3.45s	remaining: 4m 21s
13:	learn: 0.3842181	total: 3.74s	remaining: 4m 23s
14:	learn: 0.3813668	total: 4s	remaining: 4m 22s
15:	learn: 0.3791249	total: 4.27s	remaining: 4m 22s
16:	learn: 0.3769691	total: 4.54s	remaining: 4m 22s
17:	learn: 0.3751698	total: 4.81s	remaining: 4m 22s
18:	learn: 0.3735114	total: 5.07s	remaining: 4m 21

158:	learn: 0.3464729	total: 43.8s	remaining: 3m 51s
159:	learn: 0.3464194	total: 44.1s	remaining: 3m 51s
160:	learn: 0.3463399	total: 44.4s	remaining: 3m 51s
161:	learn: 0.3462454	total: 44.6s	remaining: 3m 50s
162:	learn: 0.3461603	total: 44.9s	remaining: 3m 50s
163:	learn: 0.3460737	total: 45.2s	remaining: 3m 50s
164:	learn: 0.3460254	total: 45.4s	remaining: 3m 49s
165:	learn: 0.3459591	total: 45.7s	remaining: 3m 49s
166:	learn: 0.3458915	total: 45.9s	remaining: 3m 49s
167:	learn: 0.3458286	total: 46.2s	remaining: 3m 48s
168:	learn: 0.3457772	total: 46.5s	remaining: 3m 48s
169:	learn: 0.3457172	total: 46.8s	remaining: 3m 48s
170:	learn: 0.3456445	total: 47.1s	remaining: 3m 48s
171:	learn: 0.3455899	total: 47.4s	remaining: 3m 47s
172:	learn: 0.3455163	total: 47.6s	remaining: 3m 47s
173:	learn: 0.3454366	total: 47.9s	remaining: 3m 47s
174:	learn: 0.3453928	total: 48.2s	remaining: 3m 47s
175:	learn: 0.3453480	total: 48.5s	remaining: 3m 47s
176:	learn: 0.3452393	total: 48.8s	remaining: 

312:	learn: 0.3387062	total: 1m 28s	remaining: 3m 14s
313:	learn: 0.3386585	total: 1m 28s	remaining: 3m 14s
314:	learn: 0.3386060	total: 1m 29s	remaining: 3m 13s
315:	learn: 0.3385702	total: 1m 29s	remaining: 3m 13s
316:	learn: 0.3385564	total: 1m 29s	remaining: 3m 13s
317:	learn: 0.3384815	total: 1m 30s	remaining: 3m 13s
318:	learn: 0.3384352	total: 1m 30s	remaining: 3m 12s
319:	learn: 0.3384091	total: 1m 30s	remaining: 3m 12s
320:	learn: 0.3384024	total: 1m 30s	remaining: 3m 12s
321:	learn: 0.3383668	total: 1m 31s	remaining: 3m 11s
322:	learn: 0.3383482	total: 1m 31s	remaining: 3m 11s
323:	learn: 0.3383204	total: 1m 31s	remaining: 3m 11s
324:	learn: 0.3382656	total: 1m 32s	remaining: 3m 11s
325:	learn: 0.3382148	total: 1m 32s	remaining: 3m 10s
326:	learn: 0.3381692	total: 1m 32s	remaining: 3m 10s
327:	learn: 0.3381036	total: 1m 32s	remaining: 3m 10s
328:	learn: 0.3380933	total: 1m 33s	remaining: 3m 10s
329:	learn: 0.3380678	total: 1m 33s	remaining: 3m 9s
330:	learn: 0.3380271	total: 

466:	learn: 0.3334386	total: 2m 13s	remaining: 2m 32s
467:	learn: 0.3333995	total: 2m 13s	remaining: 2m 31s
468:	learn: 0.3333756	total: 2m 13s	remaining: 2m 31s
469:	learn: 0.3333504	total: 2m 14s	remaining: 2m 31s
470:	learn: 0.3333279	total: 2m 14s	remaining: 2m 31s
471:	learn: 0.3333040	total: 2m 14s	remaining: 2m 30s
472:	learn: 0.3332709	total: 2m 15s	remaining: 2m 30s
473:	learn: 0.3332394	total: 2m 15s	remaining: 2m 30s
474:	learn: 0.3332190	total: 2m 15s	remaining: 2m 29s
475:	learn: 0.3332044	total: 2m 15s	remaining: 2m 29s
476:	learn: 0.3331749	total: 2m 16s	remaining: 2m 29s
477:	learn: 0.3331543	total: 2m 16s	remaining: 2m 29s
478:	learn: 0.3331179	total: 2m 16s	remaining: 2m 28s
479:	learn: 0.3330908	total: 2m 17s	remaining: 2m 28s
480:	learn: 0.3330657	total: 2m 17s	remaining: 2m 28s
481:	learn: 0.3330401	total: 2m 17s	remaining: 2m 27s
482:	learn: 0.3329994	total: 2m 17s	remaining: 2m 27s
483:	learn: 0.3329717	total: 2m 18s	remaining: 2m 27s
484:	learn: 0.3329543	total:

619:	learn: 0.3292645	total: 2m 57s	remaining: 1m 48s
620:	learn: 0.3292324	total: 2m 57s	remaining: 1m 48s
621:	learn: 0.3292110	total: 2m 58s	remaining: 1m 48s
622:	learn: 0.3291816	total: 2m 58s	remaining: 1m 47s
623:	learn: 0.3291460	total: 2m 58s	remaining: 1m 47s
624:	learn: 0.3291253	total: 2m 59s	remaining: 1m 47s
625:	learn: 0.3291133	total: 2m 59s	remaining: 1m 47s
626:	learn: 0.3291046	total: 2m 59s	remaining: 1m 46s
627:	learn: 0.3290966	total: 2m 59s	remaining: 1m 46s
628:	learn: 0.3290463	total: 3m	remaining: 1m 46s
629:	learn: 0.3290121	total: 3m	remaining: 1m 45s
630:	learn: 0.3289780	total: 3m	remaining: 1m 45s
631:	learn: 0.3289421	total: 3m 1s	remaining: 1m 45s
632:	learn: 0.3289134	total: 3m 1s	remaining: 1m 45s
633:	learn: 0.3288842	total: 3m 1s	remaining: 1m 44s
634:	learn: 0.3288687	total: 3m 1s	remaining: 1m 44s
635:	learn: 0.3288551	total: 3m 2s	remaining: 1m 44s
636:	learn: 0.3288180	total: 3m 2s	remaining: 1m 43s
637:	learn: 0.3287717	total: 3m 2s	remaining: 

772:	learn: 0.3255329	total: 3m 41s	remaining: 1m 5s
773:	learn: 0.3255193	total: 3m 42s	remaining: 1m 4s
774:	learn: 0.3254784	total: 3m 42s	remaining: 1m 4s
775:	learn: 0.3254616	total: 3m 42s	remaining: 1m 4s
776:	learn: 0.3254436	total: 3m 43s	remaining: 1m 4s
777:	learn: 0.3254106	total: 3m 43s	remaining: 1m 3s
778:	learn: 0.3253873	total: 3m 43s	remaining: 1m 3s
779:	learn: 0.3253469	total: 3m 43s	remaining: 1m 3s
780:	learn: 0.3253368	total: 3m 44s	remaining: 1m 2s
781:	learn: 0.3253022	total: 3m 44s	remaining: 1m 2s
782:	learn: 0.3252735	total: 3m 44s	remaining: 1m 2s
783:	learn: 0.3252558	total: 3m 45s	remaining: 1m 1s
784:	learn: 0.3252238	total: 3m 45s	remaining: 1m 1s
785:	learn: 0.3251898	total: 3m 45s	remaining: 1m 1s
786:	learn: 0.3251711	total: 3m 45s	remaining: 1m 1s
787:	learn: 0.3251384	total: 3m 46s	remaining: 1m
788:	learn: 0.3251233	total: 3m 46s	remaining: 1m
789:	learn: 0.3251210	total: 3m 46s	remaining: 1m
790:	learn: 0.3250995	total: 3m 47s	remaining: 60s
791:

929:	learn: 0.3220854	total: 4m 27s	remaining: 20.1s
930:	learn: 0.3220593	total: 4m 27s	remaining: 19.8s
931:	learn: 0.3220345	total: 4m 27s	remaining: 19.5s
932:	learn: 0.3219987	total: 4m 28s	remaining: 19.3s
933:	learn: 0.3219549	total: 4m 28s	remaining: 19s
934:	learn: 0.3219481	total: 4m 28s	remaining: 18.7s
935:	learn: 0.3219162	total: 4m 29s	remaining: 18.4s
936:	learn: 0.3218996	total: 4m 29s	remaining: 18.1s
937:	learn: 0.3218753	total: 4m 29s	remaining: 17.8s
938:	learn: 0.3218541	total: 4m 29s	remaining: 17.5s
939:	learn: 0.3218311	total: 4m 30s	remaining: 17.3s
940:	learn: 0.3218154	total: 4m 30s	remaining: 17s
941:	learn: 0.3217943	total: 4m 30s	remaining: 16.7s
942:	learn: 0.3217678	total: 4m 31s	remaining: 16.4s
943:	learn: 0.3217616	total: 4m 31s	remaining: 16.1s
944:	learn: 0.3217476	total: 4m 31s	remaining: 15.8s
945:	learn: 0.3217436	total: 4m 32s	remaining: 15.5s
946:	learn: 0.3217314	total: 4m 32s	remaining: 15.2s
947:	learn: 0.3217224	total: 4m 32s	remaining: 15s

,id,target
0,5,0.140137
1,6,0.395102
2,8,0.015622
3,9,0.217600
4,11,0.140418


In [54]:
submission.to_csv('submission_catboost_default.csv', index=False)

## Lightgbm Optuna

In [59]:
import lightgbm
x_train_cat = x_train.copy()
for cat in cat_cols:
    x_train_cat[cat] = x_train_cat[cat].astype("category")
data = lightgbm.Dataset(x_train_cat, label=y_train)

In [67]:
import optuna.integration.lightgbm as lgb
from sklearn.model_selection import KFold
params = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": 0,
        "boosting_type": "gbdt",
    }

tuner = lgb.LightGBMTunerCV(
    params, data, time_budget=60*60, folds=KFold(n_splits=5)
)

tuner.run()

[I 2021-03-03 00:18:45,153] A new study created in memory with name: no-name-ccbda61e-bde1-45f5-987b-651ea1caac13


  0%|          | 0/7 [00:00<?, ?it/s]

feature_fraction, val_score: -inf:   0%|          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008247 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007867 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008738 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008148 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing

[174]	cv_agg's auc: 0.892688 + 0.00137925
[175]	cv_agg's auc: 0.892683 + 0.00137253
[176]	cv_agg's auc: 0.892684 + 0.00137644
[177]	cv_agg's auc: 0.892683 + 0.0013844
[178]	cv_agg's auc: 0.89268 + 0.00138514
[179]	cv_agg's auc: 0.892695 + 0.00138668
[180]	cv_agg's auc: 0.892696 + 0.00138945
[181]	cv_agg's auc: 0.892703 + 0.0013939
[182]	cv_agg's auc: 0.892713 + 0.00139112
[183]	cv_agg's auc: 0.892721 + 0.00140025
[184]	cv_agg's auc: 0.892735 + 0.00139624
[185]	cv_agg's auc: 0.892737 + 0.00139569
[186]	cv_agg's auc: 0.89274 + 0.00138469
[187]	cv_agg's auc: 0.892741 + 0.001391
[188]	cv_agg's auc: 0.892746 + 0.00138869
[189]	cv_agg's auc: 0.892752 + 0.00138008
[190]	cv_agg's auc: 0.892751 + 0.00138705
[191]	cv_agg's auc: 0.892754 + 0.00138846
[192]	cv_agg's auc: 0.89276 + 0.00138671
[193]	cv_agg's auc: 0.892774 + 0.00138781
[194]	cv_agg's auc: 0.892771 + 0.00139496
[195]	cv_agg's auc: 0.892767 + 0.00139858
[196]	cv_agg's auc: 0.892767 + 0.00140476
[197]	cv_agg's auc: 0.892773 + 0.00140295

[371]	cv_agg's auc: 0.892865 + 0.00129629
[372]	cv_agg's auc: 0.892867 + 0.0012999
[373]	cv_agg's auc: 0.892864 + 0.00129753
[374]	cv_agg's auc: 0.892856 + 0.00129657
[375]	cv_agg's auc: 0.892857 + 0.0012957
[376]	cv_agg's auc: 0.892856 + 0.00130163
[377]	cv_agg's auc: 0.892863 + 0.00130009
[378]	cv_agg's auc: 0.892864 + 0.00130242
[379]	cv_agg's auc: 0.892863 + 0.00130596
[380]	cv_agg's auc: 0.892866 + 0.00130644
[381]	cv_agg's auc: 0.89287 + 0.00129857
[382]	cv_agg's auc: 0.892868 + 0.00129708
[383]	cv_agg's auc: 0.892866 + 0.00129927
[384]	cv_agg's auc: 0.892861 + 0.00130091
[385]	cv_agg's auc: 0.892855 + 0.00129791
[386]	cv_agg's auc: 0.892856 + 0.00128988
[387]	cv_agg's auc: 0.892856 + 0.00128147
[388]	cv_agg's auc: 0.892859 + 0.00128263
[389]	cv_agg's auc: 0.892867 + 0.00127625
[390]	cv_agg's auc: 0.89286 + 0.00127673
[391]	cv_agg's auc: 0.892856 + 0.00128202
[392]	cv_agg's auc: 0.892862 + 0.00127867
[393]	cv_agg's auc: 0.892859 + 0.00128722
[394]	cv_agg's auc: 0.892871 + 0.00127

[570]	cv_agg's auc: 0.892612 + 0.00134273
[571]	cv_agg's auc: 0.892611 + 0.00134196
[572]	cv_agg's auc: 0.892603 + 0.00134303
[573]	cv_agg's auc: 0.892601 + 0.00134073
[574]	cv_agg's auc: 0.892602 + 0.00134367
[575]	cv_agg's auc: 0.892602 + 0.00134352
[576]	cv_agg's auc: 0.892601 + 0.00134226
[577]	cv_agg's auc: 0.892601 + 0.00134853
[578]	cv_agg's auc: 0.892604 + 0.00134541
[579]	cv_agg's auc: 0.892602 + 0.00134709
[580]	cv_agg's auc: 0.8926 + 0.00134871
[581]	cv_agg's auc: 0.892601 + 0.00134825
[582]	cv_agg's auc: 0.892598 + 0.00135138
[583]	cv_agg's auc: 0.892597 + 0.00134291
[584]	cv_agg's auc: 0.892595 + 0.00134158
[585]	cv_agg's auc: 0.892589 + 0.00134258
[586]	cv_agg's auc: 0.892594 + 0.00134022
[587]	cv_agg's auc: 0.892602 + 0.00133878
[588]	cv_agg's auc: 0.892598 + 0.00134194
[589]	cv_agg's auc: 0.892604 + 0.00134127
[590]	cv_agg's auc: 0.892599 + 0.00133806
[591]	cv_agg's auc: 0.892603 + 0.00133494
[592]	cv_agg's auc: 0.892603 + 0.00133712
[593]	cv_agg's auc: 0.892605 + 0.001

...

[575]	cv_agg's auc: 0.893524 + 0.00125811
[576]	cv_agg's auc: 0.89353 + 0.00126196
[577]	cv_agg's auc: 0.893532 + 0.00125772
[578]	cv_agg's auc: 0.893534 + 0.00125614
[579]	cv_agg's auc: 0.893536 + 0.00125741
[580]	cv_agg's auc: 0.893537 + 0.0012566
[581]	cv_agg's auc: 0.893534 + 0.00125431
[582]	cv_agg's auc: 0.893531 + 0.00125696
[583]	cv_agg's auc: 0.893532 + 0.0012577
[584]	cv_agg's auc: 0.893535 + 0.0012573
[585]	cv_agg's auc: 0.893535 + 0.00125614
[586]	cv_agg's auc: 0.893535 + 0.00125498
[587]	cv_agg's auc: 0.893542 + 0.00125509
[588]	cv_agg's auc: 0.893542 + 0.00125605
[589]	cv_agg's auc: 0.893541 + 0.00125625
[590]	cv_agg's auc: 0.893539 + 0.00126069
[591]	cv_agg's auc: 0.893541 + 0.00125953
[592]	cv_agg's auc: 0.893544 + 0.00126091
[593]	cv_agg's auc: 0.893544 + 0.00125512
[594]	cv_agg's auc: 0.893545 + 0.00125852
[595]	cv_agg's auc: 0.893546 + 0.00126059
[596]	cv_agg's auc: 0.893546 + 0.00125401
[597]	cv_agg's auc: 0.893552 + 0.00125194
[598]	cv_agg's auc: 0.893559 + 0.00125

[775]	cv_agg's auc: 0.8938 + 0.00129478
[776]	cv_agg's auc: 0.8938 + 0.00129497
[777]	cv_agg's auc: 0.893797 + 0.0012913
[778]	cv_agg's auc: 0.893802 + 0.0012962
[779]	cv_agg's auc: 0.893806 + 0.00129385
[780]	cv_agg's auc: 0.893807 + 0.00129165
[781]	cv_agg's auc: 0.893808 + 0.00129303
[782]	cv_agg's auc: 0.893807 + 0.00129628
[783]	cv_agg's auc: 0.893805 + 0.00129333
[784]	cv_agg's auc: 0.893807 + 0.0012928
[785]	cv_agg's auc: 0.893808 + 0.00129151
[786]	cv_agg's auc: 0.893808 + 0.00129235
[787]	cv_agg's auc: 0.89381 + 0.00129548
[788]	cv_agg's auc: 0.893807 + 0.00129367
[789]	cv_agg's auc: 0.893811 + 0.00128995
[790]	cv_agg's auc: 0.893815 + 0.00128906
[791]	cv_agg's auc: 0.893813 + 0.00129073
[792]	cv_agg's auc: 0.893813 + 0.00129331
[793]	cv_agg's auc: 0.893812 + 0.00129562
[794]	cv_agg's auc: 0.893808 + 0.00129443
[795]	cv_agg's auc: 0.893808 + 0.00129216
[796]	cv_agg's auc: 0.893806 + 0.0012894
[797]	cv_agg's auc: 0.893807 + 0.00128913
[798]	cv_agg's auc: 0.893804 + 0.00128926
[

[975]	cv_agg's auc: 0.893826 + 0.00127157
[976]	cv_agg's auc: 0.89383 + 0.00127554
[977]	cv_agg's auc: 0.893827 + 0.00127848
[978]	cv_agg's auc: 0.893829 + 0.00127881
[979]	cv_agg's auc: 0.893831 + 0.00127461
[980]	cv_agg's auc: 0.893833 + 0.00127422
[981]	cv_agg's auc: 0.893831 + 0.00127518
[982]	cv_agg's auc: 0.893831 + 0.00127546
[983]	cv_agg's auc: 0.893832 + 0.0012778
[984]	cv_agg's auc: 0.893836 + 0.00127981
[985]	cv_agg's auc: 0.893839 + 0.00128144
[986]	cv_agg's auc: 0.893841 + 0.00128
[987]	cv_agg's auc: 0.893841 + 0.00127673
[988]	cv_agg's auc: 0.89384 + 0.00127749
[989]	cv_agg's auc: 0.893838 + 0.00127696
[990]	cv_agg's auc: 0.893841 + 0.00127604
[991]	cv_agg's auc: 0.893839 + 0.00127376
[992]	cv_agg's auc: 0.89384 + 0.00127164
[993]	cv_agg's auc: 0.893836 + 0.00127033
[994]	cv_agg's auc: 0.893832 + 0.00127364
[995]	cv_agg's auc: 0.893826 + 0.00127438
[996]	cv_agg's auc: 0.893824 + 0.00127331
[997]	cv_agg's auc: 0.893824 + 0.00126963
[998]	cv_agg's auc: 0.893824 + 0.00126889



min_data_in_leaf, val_score: 0.893979:  80%|########  | 4/5 [04:04<00:48, 48.80s/it]

min_data_in_leaf, val_score: 0.893979: 100%|##########| 5/5 [04:04<00:00, 48.91s/it]

[1000]	cv_agg's auc: 0.893822 + 0.0012669


In [68]:
tuner.best_score

0.893978668987455

In [69]:
tuner.best_params

{'objective': 'binary',
 'metric': 'auc',
 'verbosity': 0,
 'boosting_type': 'gbdt',
 'feature_pre_filter': False,
 'lambda_l1': 8.933237749923594,
 'lambda_l2': 4.324012686862672e-06,
 'num_leaves': 12,
 'feature_fraction': 0.44800000000000006,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20}

In [73]:
lightgbm_tuned_pipe = Pipeline([("trans", CategoricalTransform(cat_cols)),
                          ("lgbm", LGBMClassifier(n_jobs=-2, **tuner.best_params))])

In [74]:
submission = submit(lightgbm_tuned_pipe)
submission.head()

[LightGBM] [Warning] feature_fraction is set=0.44800000000000006, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.44800000000000006
[LightGBM] [Warning] lambda_l1 is set=8.933237749923594, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.933237749923594
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] lambda_l2 is set=4.324012686862672e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.324012686862672e-06
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] feature_fraction is set=0.44800000000000006, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.44800000000000006
[LightGBM] [Warning] lambda_l1 is set=8.933237749923594, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.933237749923594
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1

,id,target
0,5,0.141023
1,6,0.339664
2,8,0.042516
3,9,0.204505
4,11,0.106371


In [75]:
submission.to_csv('submission_lightgbm_optuna.csv', index=False)

Worse than default catboost!